In [54]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities.head()

,Metropolitan area,Country,Pop. rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3],11,Red Bulls New York City FC,—
1,Los Angeles,United States,2,13310447,8,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks,10,Galaxy Los Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [53]:
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities

IndexError: positional indexers are out-of-bounds

In [ ]:
def split_team_names(string):
    split = string.split(' ')
    if len(split) > 1:
        new_split = []
        while split != []:
            if split[0][-1] == 's': 
                new_split.append(split.pop(0))
                if split == []: break
            if split[0][-1] != 's': 
                if len(split) == 1: new_split.append(split.pop(0))
                else: new_split.append(' '.join([split.pop(0), split.pop(0)]))
        return new_split
    else: return split

s = ''
split_team_names()

In [ ]:
def match_teams(region_team_string, team_to_match):
    for region_team in split_team_names(region_team_string): 
        if region_team in team_to_match: return region_team

match_teams()

In [23]:
nhl = nhl_df[nhl_df['year'] == 2018]

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL
6,Montreal Canadiens,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL
7,Ottawa Senators,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL
8,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL
9,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,2018,NHL


In [24]:
nhl = nhl.drop(nhl.index[nhl['team'].str.contains('Division')])

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL
6,Montreal Canadiens,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL
7,Ottawa Senators,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL
8,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL
10,Washington Capitals*,82,49,26,7,105,.640,259,239,0.21,-0.04,.585,46,2018,NHL
11,Pittsburgh Penguins*,82,47,29,6,100,.610,272,250,0.23,-0.04,.573,45,2018,NHL


In [50]:
nhl['team'] = nhl['team'].str.replace('*', '')

In [51]:
nhl['W/L ratio'] = nhl['W'].apply(int) / (nhl['W'].apply(int) + nhl['L'].apply(int))

In [52]:
all_pop = cities.set_index('Metropolitan area')[['Population (2016 est.)[8]','NHL']]

In [ ]:
all_pop['NHL'] = all_pop['NHL'].replace("—", "").str.replace("\[.*\]","")

In [ ]:
global nhl_pop
nhl_pop = all_pop[all_pop['NHL'].str.contains('\w+')]

In [ ]:
def match_metro(row): return nhl_pop['NHL'].apply(lambda x: match_teams(x, row)).dropna().index[0]
match_metro()

In [ ]:
nhl['Metropolitan area'] = nhl['team'].apply(match_metro)
city_ratios = nhl[['W/L ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
nhl_pop = nhl_pop.merge(city_ratios, on='Metropolitan area')


In [2]:
def nhl_correlation(): 
    population_by_region = nhl_pop['Population (2016 est.)[8]'].apply(int) # pass in metropolitan area population from cities
    win_loss_by_region = nhl_pop['W/L ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nhl_correlation()

IndexError: index 0 is out of bounds for axis 0 with size 0

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    nba = nba_df[lambda x: x['year'] == 2018]
    nba['team'] = nba['team'].str.replace('*', '').str.replace("\(.*\)","")
    nba['W/L ratio'] = nba['W'].apply(int) / (nba['W'].apply(int) + nba['L'].apply(int))
    
    all_pop = cities.set_index('Metropolitan area')[['Population (2016 est.)[8]','NBA']]
    all_pop['NBA'] = all_pop['NBA'].replace("—", "").str.replace("\[.*\]","")
    global nba_pop
    nba_pop = all_pop[all_pop['NBA'].str.contains('\w+')]
    
    def match_metro(row): return nba_pop['NBA'].apply(lambda x: match_teams(x, row)).dropna().index[0]
    nba['Metropolitan area'] = nba['team'].apply(match_metro)
    city_ratios = nba[['W/L ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    nba_pop = nba_pop.merge(city_ratios, on='Metropolitan area')
    
    population_by_region = nba_pop['Population (2016 est.)[8]'].apply(int) # pass in metropolitan area population from cities
    win_loss_by_region = nba_pop['W/L ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


nba_correlation()

(-0.17657160252844617, 0.36874741604463)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    mlb = mlb_df[lambda x: x['year'] == 2018]
    mlb['W/L ratio'] = mlb['W-L%']
        
    all_pop = cities.set_index('Metropolitan area')[['Population (2016 est.)[8]','MLB']]
    all_pop['MLB'] = all_pop['MLB'].replace("—", "").str.replace("\[.*\]","")
    global mlb_pop
    mlb_pop = all_pop[all_pop['MLB'].str.contains('\w+')]
    
    def match_metro(row): return mlb_pop['MLB'].apply(lambda x: match_teams(x, row)).dropna().index[0]
    mlb['Metropolitan area'] = mlb['team'].apply(match_metro)
    city_ratios = mlb[['W/L ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    mlb_pop = mlb_pop.merge(city_ratios, on='Metropolitan area')
    
    population_by_region = mlb_pop['Population (2016 est.)[8]'].apply(int) # pass in metropolitan area population from cities
    win_loss_by_region = mlb_pop['W/L ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

(0.1500373747540949, 0.46442827201123393)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    nfl = nfl_df[lambda x: x['year'] == 2018]
    nfl = nfl.drop(nfl.index[nfl['team'].str.contains('FC')])
    nfl['team'] = nfl['team'].str.replace('[*+]', '')
    nfl['W/L ratio'] = nfl['W'].apply(int) / (nfl['W'].apply(int) + nfl['L'].apply(int))
    
    all_pop = cities.set_index('Metropolitan area')[['Population (2016 est.)[8]','NFL']]
    all_pop['NFL'] = all_pop['NFL'].str.replace("\[.*\]","")
    global nfl_pop
    nfl_pop = all_pop[all_pop['NFL'].str.contains('\w+')]
        
    def match_metro(row): return nfl_pop['NFL'].apply(lambda x: match_teams(x, row)).dropna().index[0]
    nfl['Metropolitan area'] = nfl['team'].apply(match_metro)
    city_ratios = nfl[['W/L ratio', 'Metropolitan area']].groupby('Metropolitan area').mean()
    nfl_pop = nfl_pop.merge(city_ratios, on='Metropolitan area')
        
    population_by_region = nfl_pop['Population (2016 est.)[8]'].apply(int) # pass in metropolitan area population from cities
    win_loss_by_region = nfl_pop['W/L ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

(0.004922112149349414, 0.9797833458363694)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [28]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    pairs = ['NFL_NBA', 'NFL_NHL', 'NFL_MLB', 'NBA_NHL', 'NBA_MLB', 'NHL_MLB']
        
    NFL_NBA = nfl_pop.merge(nba_pop, on='Metropolitan area')
    NFL_NHL = nfl_pop.merge(nhl_pop, on='Metropolitan area')
    NFL_MLB = nfl_pop.merge(mlb_pop, on='Metropolitan area')
    NBA_NHL = nba_pop.merge(nhl_pop, on='Metropolitan area')
    NBA_MLB = nba_pop.merge(mlb_pop, on='Metropolitan area')
    NHL_MLB = nhl_pop.merge(mlb_pop, on='Metropolitan area')
    
    NFL_NBA_stat = stats.ttest_rel(NFL_NBA['W/L ratio_x'], NFL_NBA['W/L ratio_y'])
    NFL_NHL_stat = stats.ttest_rel(NFL_NHL['W/L ratio_x'], NFL_NHL['W/L ratio_y'])
    NFL_MLB_stat = stats.ttest_rel(NFL_MLB['W/L ratio_x'], NFL_MLB['W/L ratio_y'])
    NBA_NHL_stat = stats.ttest_rel(NBA_NHL['W/L ratio_x'], NBA_NHL['W/L ratio_y'])
    NBA_MLB_stat = stats.ttest_rel(NBA_MLB['W/L ratio_x'], NBA_MLB['W/L ratio_y'])
    NHL_MLB_stat = stats.ttest_rel(NHL_MLB['W/L ratio_x'], NHL_MLB['W/L ratio_y'])
    
    p_values.loc["NFL", "NBA"] = NFL_NBA_stat.pvalue
    p_values.loc["NBA", "NFL"] = NFL_NBA_stat.pvalue
    p_values.loc["NFL", "NHL"] = NFL_NHL_stat.pvalue
    p_values.loc["NHL", "NFL"] = NFL_NHL_stat.pvalue
    p_values.loc["NFL", "MLB"] = NFL_MLB_stat.pvalue
    p_values.loc["MLB", "NFL"] = NFL_MLB_stat.pvalue
    p_values.loc["NBA", "NHL"] = NBA_NHL_stat.pvalue
    p_values.loc["NHL", "NBA"] = NBA_NHL_stat.pvalue
    p_values.loc["NBA", "MLB"] = NBA_MLB_stat.pvalue
    p_values.loc["MLB", "NBA"] = NBA_MLB_stat.pvalue
    p_values.loc["NHL", "MLB"] = NHL_MLB_stat.pvalue
    p_values.loc["MLB", "NHL"] = NHL_MLB_stat.pvalue
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802499
NBA,0.941792,NaN,0.022297,0.951629
NHL,0.030883,0.022297,NaN,0.000703
MLB,0.802499,0.951629,0.000703,NaN
